# Forecast Analysis
## Imports

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts

from io import StringIO

import os

## Load data

In [11]:
prediction_data = pd.DataFrame()

for file in os.listdir("prediction-data"):
    if "024" in file: # ONLY RUN ON 24 HOURS
        with open(f"prediction-data/{file}") as f:
            data = f.read()
            curr_data = pd.read_csv(StringIO(data), low_memory=False, sep=" ")
            prediction_data = pd.concat([prediction_data, curr_data])
    
prediction_data.columns = prediction_data.columns.str.strip()
prediction_data.set_index("date", inplace=True)
prediction_data.index = pd.to_datetime(prediction_data.index)
prediction_data: pd.DataFrame = prediction_data.apply(pd.to_numeric, errors='coerce')
prediction_data.loc[:, "loc_nr"] = prediction_data["loc_nr"].astype(str).str.slice(1).astype(int)
prediction_data.set_index("loc_nr", append=True, inplace=True)
prediction_data.loc[:, "mean_pred"] = prediction_data[[f"E{i + 1}" for i in range(50)]].mean(axis=1)
prediction_data.loc[:, "median_pred"] = prediction_data[[f"E{i + 1}" for i in range(50)]].median(axis=1)
prediction_data.loc[:, "min_pred"] = prediction_data[[f"E{i + 1}" for i in range(50)]].min(axis=1)
prediction_data.loc[:, "max_pred"] = prediction_data[[f"E{i + 1}" for i in range(50)]].max(axis=1)
prediction_data.loc[:, "pred_std"] = prediction_data[[f"E{i + 1}" for i in range(50)]].std(axis=1)
prediction_data.loc[:, "pred_skew"] = prediction_data[[f"E{i + 1}" for i in range(50)]].skew(axis=1)
prediction_data.loc[:, "pred_kurt"] = prediction_data[[f"E{i + 1}" for i in range(50)]].kurt(axis=1)
prediction_data.loc[:, "mode_pred"] = prediction_data[[f"E{i + 1}" for i in range(50)]].mode(axis=1, numeric_only=True).mean(axis=1)
prediction_data.loc[:, "mode2_pred"] = prediction_data[[f"E{i + 1}" for i in range(50)]].mode(axis=1, numeric_only=True).median(axis=1)
prediction_data.loc[:, "mean_det_pred"] = prediction_data[["mean_pred", "det_run"]].mean(axis=1)
prediction_data.to_hdf("prediction_data.hdf5", "prediction_data_24h")
prediction_data

,,det_run,E1,E2,E3,E4,E5,E6,E7,E8,E9,...,mean_pred,median_pred,min_pred,max_pred,pred_std,pred_skew,pred_kurt,mode_pred,mode2_pred,mean_det_pred
date,loc_nr,,,,,,,,,,,,,,,,,,,,,
2016-04-01,260,0,0,0,0,0,0,0,0,0,0,...,0.16,0.0,0,3,0.509502,4.129082,19.937964,0.000000,0.0,0.08
2016-04-02,260,1,1,5,2,3,3,1,1,2,1,...,2.66,2.0,0,16,3.360090,2.668093,7.322484,2.000000,2.0,1.83
2016-04-03,260,34,20,8,22,19,26,19,28,12,25,...,25.90,22.0,8,75,14.570378,1.523376,2.709633,19.333333,19.0,29.95
2016-04-04,260,17,44,54,29,29,44,54,9,61,22,...,33.40,29.0,7,88,16.099182,1.028039,1.597512,36.200000,29.0,25.20
2016-04-05,260,30,18,8,24,58,36,41,30,43,25,...,31.30,31.5,8,62,15.708116,0.159005,-1.070315,8.000000,8.0,30.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-26,240,0,0,0,0,0,0,0,0,0,0,...,0.00,0.0,0,0,0.000000,0.000000,0.000000,0.000000,0.0,0.00
2021-04-27,240,0,0,0,0,2,0,0,1,0,1,...,0.36,0.0,0,3,0.692820,2.062579,4.056756,0.000000,0.0,0.18
2021-04-28,240,41,27,10,38,29,5,31,14,32,20,...,22.82,21.0,3,59,14.342302,0.655763,-0.160606,14.000000,14.0,31.91


In [12]:
true_weather = pd.read_hdf("weather_data.hdf5", "measured_data")
rainfall: pd.DataFrame = true_weather["RH-fix"]
daily_rainfall = rainfall.groupby([pd.Grouper(freq="D", level=0), rainfall.index.get_level_values(1)]).sum()
daily_rainfall.index.rename(["date", "loc_nr"], inplace=True)
daily_rainfall = daily_rainfall * 10
daily_rainfall

date        loc_nr
1951-01-01  240         0.0
            260        23.0
            310         0.0
1951-01-02  240         0.0
            260        25.0
                      ...  
2022-06-05  260       259.0
            310       209.0
2022-06-06  240        44.0
            260       114.0
            310       109.0
Name: RH-fix, Length: 78267, dtype: float64

## Compare data

In [13]:
combined_data = prediction_data.join(daily_rainfall)
combined_data.loc[:, "det-difference"] = combined_data["RH-fix"] - combined_data["det_run"]
combined_data.loc[:, "pred-difference"] = combined_data["RH-fix"] - combined_data["mean_pred"]
combined_data.loc[:, "med-difference"] = combined_data["RH-fix"] - combined_data["median_pred"]
combined_data.loc[:, "mode-difference"] = combined_data["RH-fix"] - combined_data["mode_pred"]
combined_data.loc[:, "mode2-difference"] = combined_data["RH-fix"] - combined_data["mode2_pred"]
combined_data.loc[:, "dp-difference"] = combined_data["RH-fix"] - combined_data["mean_det_pred"]
combined_data

,,det_run,E1,E2,E3,E4,E5,E6,E7,E8,E9,...,mode_pred,mode2_pred,mean_det_pred,RH-fix,det-difference,pred-difference,med-difference,mode-difference,mode2-difference,dp-difference
date,loc_nr,,,,,,,,,,,,,,,,,,,,,
2016-04-01,260,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.0,0.08,0.0,0.0,-0.16,0.0,0.000000,0.0,-0.08
2016-04-02,260,1,1,5,2,3,3,1,1,2,1,...,2.000000,2.0,1.83,0.0,-1.0,-2.66,-2.0,-2.000000,-2.0,-1.83
2016-04-03,260,34,20,8,22,19,26,19,28,12,25,...,19.333333,19.0,29.95,97.0,63.0,71.10,75.0,77.666667,78.0,67.05
2016-04-04,260,17,44,54,29,29,44,54,9,61,22,...,36.200000,29.0,25.20,63.0,46.0,29.60,34.0,26.800000,34.0,37.80
2016-04-05,260,30,18,8,24,58,36,41,30,43,25,...,8.000000,8.0,30.65,14.0,-16.0,-17.30,-17.5,6.000000,6.0,-16.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-26,240,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.0,0.00
2021-04-27,240,0,0,0,0,2,0,0,1,0,1,...,0.000000,0.0,0.18,0.0,0.0,-0.36,0.0,0.000000,0.0,-0.18
2021-04-28,240,41,27,10,38,29,5,31,14,32,20,...,14.000000,14.0,31.91,0.0,-41.0,-22.82,-21.0,-14.000000,-14.0,-31.91


In [14]:
smse_det = np.sqrt((combined_data["det-difference"] ** 2).mean())
smse_pred = np.sqrt((combined_data["pred-difference"] ** 2).mean())
smse_med_pred = np.sqrt((combined_data["med-difference"] ** 2).mean())
smse_mode_pred = np.sqrt((combined_data["mode-difference"] ** 2).mean())
smse_mode2_pred = np.sqrt((combined_data["mode2-difference"] ** 2).mean())
smse_dp = np.sqrt((combined_data["dp-difference"] ** 2).mean())
f"{smse_det=}, {smse_pred=}, {smse_med_pred=}, {smse_mode_pred=}, {smse_mode2_pred=}, {smse_dp=}"

'smse_det=30.887370389783708, smse_pred=29.069563707767045, smse_med_pred=29.03947494568891, smse_mode_pred=29.888378105822216, smse_mode2_pred=29.84939962012537, smse_dp=29.461948449670306'

In [15]:
mae_det = np.abs(combined_data["det-difference"]).mean()
mae_pred = np.abs(combined_data["pred-difference"]).mean()
mae_med_pred = np.abs(combined_data["med-difference"]).mean()
mae_mode_pred = np.abs(combined_data["mode-difference"]).mean()
mae_mode2_pred = np.abs(combined_data["mode-difference"]).mean()
mae_dp = np.abs(combined_data["dp-difference"]).mean()
f"{mae_det=}, {mae_pred=}, {mae_med_pred=}, {mae_mode_pred=}, {mae_mode2_pred=}, {mae_dp=}"

'mae_det=13.318059299191376, mae_pred=12.933516621743061, mae_med_pred=12.380233602875112, mae_mode_pred=12.161443614686199, mae_mode2_pred=12.161443614686199, mae_dp=12.868300089847255'

In [16]:
smae_det = (np.abs(combined_data["det-difference"])/((np.abs(combined_data["det_run"]) + np.abs(combined_data["RH-fix"])/2))).mean()
smae_pred = (np.abs(combined_data["pred-difference"])/((np.abs(combined_data["mean_pred"]) + np.abs(combined_data["RH-fix"])/2))).mean()
smae_med_pred = (np.abs(combined_data["med-difference"])/((np.abs(combined_data["median_pred"]) + np.abs(combined_data["RH-fix"])/2))).mean()
smae_mode_pred = (np.abs(combined_data["mode-difference"])/((np.abs(combined_data["mode_pred"]) + np.abs(combined_data["RH-fix"])/2))).mean()
smae_mode2_pred = (np.abs(combined_data["mode2-difference"])/((np.abs(combined_data["mode2_pred"]) + np.abs(combined_data["RH-fix"])/2))).mean()
smae_dp = (np.abs(combined_data["dp-difference"])/((np.abs(combined_data["mean_det_pred"]) + np.abs(combined_data["RH-fix"])/2))).mean()
f"{smae_det=}, {smae_pred=}, {smae_med_pred=}, {smae_mode_pred=}, {smae_mode_pred=}, {smae_dp=}"

'smae_det=0.6274840728670519, smae_pred=0.6713785767854612, smae_med_pred=0.61377723656133, smae_mode_pred=0.6294526294393507, smae_mode_pred=0.6294526294393507, smae_dp=0.6758446985382126'

## Correlation deterministic and predicted

In [17]:
prediction_data[["mean_pred", "det_run", "median_pred", "mode_pred", "mode2_pred"]].corr()

,mean_pred,det_run,median_pred,mode_pred,mode2_pred
mean_pred,1.000000,0.955258,0.996995,0.977414,0.975106
det_run,0.955258,1.000000,0.955954,0.942353,0.939742
median_pred,0.996995,0.955954,1.000000,0.983236,0.981600
mode_pred,0.977414,0.942353,0.983236,1.000000,0.998856
mode2_pred,0.975106,0.939742,0.981600,0.998856,1.000000
